<a href="https://colab.research.google.com/github/dnguyend/MiscCollection/blob/main/MiscStiefelLogTest/misc_stiefel_frechet/JuliaStiefelLogMiminizeLBFGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Light LBFGS solver for Stiefel Logarithm with Frechet derivatives
# The dependency on Optim has been removed.
* We implemented in Julia a light version of LBFGS adapting [minFunc](https://www.cs.ubc.ca/~schmidtm/Software/minFunc.html). This solver, with a simple Wolfe line search is less robust than (Optim + More'- Thuente line search), but seems sufficient for our purpose. 
* We moved the test result directory to a new folder in the repository [MiscCollection](https://github.com/dnguyend/MiscCollection/tree/main/MiscStiefelLogTest/misc_stiefel_frechet) to keep Manifolds.jl clean.
* One modification versus the method described in [[1]](https://link.springer.com/article/10.1007/s10957-022-02012-3) is we call LBFGS to optimize up to a distance *pretol*. After that we just run a simple descent based on the distance. This modification help speed up the algorithm.
* The algorithm, with customized LBFGS compares well with the log function for StiefelSubmersionMetric. However we notice when both the manifold size and distance are large, the production version of Log seems to have an advantage.
  * For longer distance, a pretol of 1e-7 produces better result (previously we used 1e-5 with the Optim package)

[1] Nguyen, D. Closed-form Geodesics and Optimization for Riemannian Logarithms of Stiefel and Flag Manifolds. J Optim Theory Appl 194, 142–166 (2022). https://doi.org/10.1007/s10957-022-02012-3



## The workbook could be viewed as is, but could also be run in a colab environment.
## Instructions to run the workbook

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab 

1. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). **This takes a couple of minutes.**
2. **Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.**

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 1 and 2.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 1 and 2.

In [ ]:
%%shell
# set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  GPU_LIST=`nvidia-smi -L 2> /dev/null`
  if [ "$?" -eq "0" ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2022-10-29 02:44:41 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake-avx512)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools
using LinearAlgebra
using Pkg
Pkg.add(url="https://github.com/dnguyend/Manifolds.jl")
using Manifolds

     Cloning git-repo `https://github.com/dnguyend/Manifolds.jl`
    Updating git-repo `https://github.com/dnguyend/Manifolds.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ─────────────────────── v0.5.1
   Installed HypergeometricFunctions ──────── v0.3.11
   Installed PDMats ───────────────────────── v0.11.16
   Installed StatsFuns ────────────────────── v1.0.1
   Installed ZygoteRules ──────────────────── v0.2.2
   Installed RecursiveArrayTools ──────────── v2.32.1
   Installed ArnoldiMethod ────────────────── v0.2.0
   Installed Kronecker ────────────────────── v0.5.3
   Installed Einsum ───────────────────────── v0.4.1
   Installed ArrayInterfaceCore ───────────── v0.1.23
   Installed HybridArrays ─────────────────── v0.4.13
   Installed StaticArraysCore ─────────────── v1.4.0
   Installed LinearMaps ───────────────────── v3.8.0
   Installed StaticArrays ─────────────────── v1.5.9
   Installed Rmath_jll ──────

In [3]:
using Printf
using Random


## Riemannian Logarithm using local minimize_lbfgs
* We implement the function Manifolds.log_lbfgs, structured after Manifolds.log
* A warmup run:

In [4]:
function linf(mat)
  return maximum(abs.(mat))
end  

function randpoint(M)
    return project(M, randn(representation_size(M)))
end

function randvec(M, p)
    # generate unit vector
    X = project(M, p, randn(representation_size(M)))
    X ./= sqrt(inner(M, p, X, X))
    return X
end

# do a warm up run
n = 5
k = 3
α = .2

M = MetricManifold(Stiefel(n, k), StiefelSubmersionMetric(α))
p = randpoint(M)
X = randvec(M, p)

q = exp(M, p, X)

XA = Manifolds.log(M, p, q)
XB = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10)
display(linf(XA - XB))

1.2415143357813463e-9

Now compare and benchmark for a bigger example. Generated distance is $0.5\pi$

In [5]:
 n = 1000
 k = 200

 # We test for all α >-1.
 α = 3*rand() - .9
 
 M = MetricManifold(Stiefel(n, k), StiefelSubmersionMetric(α))
 p = randpoint(M)
 X = randvec(M, p)

 # example with distance 0.5pi
 ft = .5
 q = exp(M, p, ft*pi*X)

 XOld = Manifolds.log(M, p, q)
 XF = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, max_itr=1000, pretol=1e-3)
 display(linf(XOld - XF))
 display(linf(XOld - ft*pi*X))
 display(linf(XF - ft*pi*X))

7.813363671338447e-13

2.96115562625765e-13

7.016275581361864e-13

In [6]:
 display(@benchmark Manifolds.log(M, p, q))
 display(@benchmark Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, max_itr=1000, pretol=1e-3))

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  565.596 ms … 917.388 ms  ┊ GC (min … max): 2.30% … 1.84%
 Time  (median):     593.636 ms               ┊ GC (median):    2.64%
 Time  (mean ± σ):   694.632 ms ± 158.773 ms  ┊ GC (mean ± σ):  2.49% ± 0.90%

  ▁█  ▁▁                                            ▁   ▁     ▁  
  ██▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁█ ▁
  566 ms           Histogram: frequency by time          917 ms <

 Memory estimate: 275.36 MiB, allocs estimate: 2332.

BenchmarkTools.Trial: 14 samples with 1 evaluation.
 Range (min … max):  354.436 ms … 406.992 ms  ┊ GC (min … max): 3.38% … 2.74%
 Time  (median):     379.768 ms               ┊ GC (median):    2.18%
 Time  (mean ± σ):   379.759 ms ±  16.262 ms  ┊ GC (mean ± σ):  2.39% ± 0.57%

  █   █    █ █     █         ██ █   █        █ ██    █        █  
  █▁▁▁█▁▁▁▁█▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁██▁█▁▁▁█▁▁▁▁▁▁▁▁█▁██▁▁▁▁█▁▁▁▁▁▁▁▁█ ▁
  354 ms           Histogram: frequency by time          407 ms <

 Memory estimate: 206.74 MiB, allocs estimate: 2436.

* Another example. Generated distance is $2\pi$

In [22]:
 Random.seed!(1)
 n = 500
 k = 200

 α = 3*rand() - .9
 
 M = MetricManifold(Stiefel(n, k), StiefelSubmersionMetric(α))
 p = randpoint(M)
 X = randvec(M, p)

 ft = 2.
 q = exp(M, p, ft*pi*X)

 XOld = Manifolds.log(M, p, q)
 XF = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-7)
 display(linf(XOld - XF))
 display(linf(XOld - ft*pi*X))
 display(linf(XF - ft*pi*X))
 display(@benchmark Manifolds.log(M, p, q))
 display(@benchmark Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-7))

6.049236198762964e-11

4.669630654374757e-11

1.859764599265734e-11

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.343 s …   1.367 s  ┊ GC (min … max): 1.61% … 2.04%
 Time  (median):     1.346 s              ┊ GC (median):    1.66%
 Time  (mean ± σ):   1.350 s ± 11.358 ms  ┊ GC (mean ± σ):  1.71% ± 0.24%

  █         ▁                                             ▁  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.34 s         Histogram: frequency by time        1.37 s <

 Memory estimate: 484.53 MiB, allocs estimate: 4454.

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.317 s …    1.674 s  ┊ GC (min … max): 2.75% … 20.66%
 Time  (median):     1.373 s               ┊ GC (median):    4.15%
 Time  (mean ± σ):   1.435 s ± 163.516 ms  ┊ GC (mean ± σ):  8.52% ±  8.71%

  █   █        █                                           █  
  █▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.32 s         Histogram: frequency by time         1.67 s <

 Memory estimate: 713.85 MiB, allocs estimate: 10097.

# At $4\pi$

In [23]:
 # Random.seed!(0)
 ft = 4.
 q = exp(M, p, ft*pi*X)

 XOld = Manifolds.log(M, p, q)
 XF = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-7)
 display(linf(XOld - XF))
 display(linf(XOld - ft*pi*X))
 display(linf(XF - ft*pi*X))
 display(@benchmark Manifolds.log(M, p, q))
 display(@benchmark Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-7))

1.2365749223197664e-10

1.0709664405306185e-10

9.866330951020796e-11

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.685 s …    3.687 s  ┊ GC (min … max): 1.32% … 1.36%
 Time  (median):     3.186 s               ┊ GC (median):    1.34%
 Time  (mean ± σ):   3.186 s ± 708.352 ms  ┊ GC (mean ± σ):  1.34% ± 0.03%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.69 s         Histogram: frequency by time         3.69 s <

 Memory estimate: 865.25 MiB, allocs estimate: 7906.

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.957 s …   2.988 s  ┊ GC (min … max): 1.96% … 3.66%
 Time  (median):     2.972 s              ┊ GC (median):    2.81%
 Time  (mean ± σ):   2.972 s ± 22.004 ms  ┊ GC (mean ± σ):  2.81% ± 1.21%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.96 s         Histogram: frequency by time        2.99 s <

 Memory estimate: 1.49 GiB, allocs estimate: 20789.

*  We test extensively using the script [test_log_frechet.jl](https://github.com/dnguyend/MiscCollection/blob/main/MiscStiefelLogTest/misc_stiefel_frechet/test_log_frechet.jl) to generate manifolds of different size, alpha and distance. We save the result in csv files, max_05 and max_05_pretol5 corresponding to distance at most $0.5\pi$, max_20 corresponds to distance at most $2\pi$ and max_40 corresponds to distance at most $4\pi$
We put data in data frames and summarize the results:

In [24]:

download("https://raw.githubusercontent.com/dnguyend/MiscCollection/main/MiscStiefelLogTest/misc_stiefel_frechet/test_with_local_lbfgs/max_05.csv", "max_05.csv")
download("https://raw.githubusercontent.com/dnguyend/MiscCollection/main/MiscStiefelLogTest/misc_stiefel_frechet/test_with_local_lbfgs/max_05_pretol5.csv", "max_05_pretol5.csv")
download("https://raw.githubusercontent.com/dnguyend/MiscCollection/main/MiscStiefelLogTest/misc_stiefel_frechet/test_with_local_lbfgs/max_20.csv", "max_20.csv")
download("https://raw.githubusercontent.com/dnguyend/MiscCollection/main/MiscStiefelLogTest/misc_stiefel_frechet/test_with_local_lbfgs/max_40.csv", "max_40.csv")

"max_40.csv"

In [9]:
Pkg.add("DataFrames")
Pkg.add("CSV")
using DataFrames, CSV

   Resolving package versions...
┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 7, "Couldn't connect to server", Downloads.Response("https", "https://in.pkg.julialang.org/registries", 301, "HTTP/2 301", ["server" => "Varnish", "retry-after" => "0", "location" => "https://in.pkg.julialang.org/registries", "x-geo-continent" => "AS", "x-geo-country" => "SG", "x-geo-region" => "01", "accept-ranges" => "bytes", "date" => "Sat, 29 Oct 2022 02:56:23 GMT", "via" => "1.1 varnish", "x-served-by" => "cache-qpg1248-QPG", "x-cache" => "HIT", "x-cache-hits" => "0", "x-timer" => "S1667012184.987835,VS0,VE1", "content-length" => "0"]))
└ @ Pkg.Registry /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Registry/Registry.jl:68
   Installed PooledArrays ─────── v1.4.2
   Installed InvertedIndices ──── v1.1.0
   Installed Crayons ──────────── v4.1.1
   Installe

* Comparing data, errOld is error using the old log, errNew is using the new log, timeOld is time using the old log, time new using new log. We also compare the norm. We will see sometime the log is better than the data generating norm (colum normGEN, ie if we generate point q using vector  $f\pi X$, old norm is $fp\pi |X|_{Riemann}$.

In [25]:
# at distance at most 0.5pi
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m05 = DataFrame(CSV.File("max_05.csv", header=false))
rename!(m05, header)
display(show(m05, allrows=true, allcols=true))
describe(m05)

50×10 DataFrame
 Row │ n        k        alpha       errOld       errNew       timeOld      timeNew      normOld   normNew   normGen  
     │ Float64  Float64  Float64     Float64      Float64      Float64      Float64      Float64   Float64   Float64  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    20.0      3.0   1.68642    2.21184e-10  1.49331e-12  0.000803736  0.000239151  0.574269  0.574269  0.574269
   2 │    95.0      9.0   0.0392704  9.16708e-12  3.78308e-14  0.00115071   0.00071861   0.435339  0.435339  0.435339
   3 │   818.0    631.0   0.213541   1.77093e-12  3.86826e-14  2.57352      1.67809      0.439288  0.439288  0.439288
   4 │   110.0     76.0   1.957      1.42964e-10  2.94126e-12  0.0453113    0.0277302    1.09439   1.09439   1.09439
   5 │    56.0     25.0  -0.826732   1.66782e-10  1.76192e-12  0.00592416   0.00392369   0.390783  0.390783  0.390783
   6 │   185.0     42.0  -0.0805532  1

nothing

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,4.27916e-10,1.08961e-13,3.17228e-11,3.00824e-9,0,Float64
5,errNew,2.43448e-11,3.27537e-14,1.98678e-12,4.4555e-10,0,Float64
6,timeOld,0.239635,0.000601534,0.00458483,3.49566,0,Float64
7,timeNew,0.144402,0.00017542,0.00267137,2.05929,0,Float64
8,normOld,0.795729,0.187572,0.746433,1.55651,0,Float64
9,normNew,0.795729,0.187572,0.746433,1.55651,0,Float64


In [26]:
# same distance 0.5pi, but pretol=1e-5 -
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m05p5 = DataFrame(CSV.File("max_05_pretol5.csv", header=false))
rename!(m05p5, header)
describe(m05p5)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,4.27916e-10,1.08961e-13,3.17228e-11,3.00824e-9,0,Float64
5,errNew,2.30014e-11,2.1155e-14,2.74431e-12,4.56195e-10,0,Float64
6,timeOld,0.238219,0.00060856,0.00463792,3.4883,0,Float64
7,timeNew,0.182671,0.000210703,0.00285558,2.78086,0,Float64
8,normOld,0.795729,0.187572,0.746433,1.55651,0,Float64
9,normNew,0.795729,0.187572,0.746433,1.55651,0,Float64


# For distance at $2\pi$ we use pretol 1e-7. The result still looks pretty good although there are several examples where the new method underperforms in term of error, ie it is further from the target point than the old method. But it out performs in some other examples, and timewise is still fast. 

* the result seems to be worse than using Optim.

In [27]:
# distance at most 2pi
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m20 = DataFrame(CSV.File("max_20.csv", header=false))
rename!(m20, header)
display(show(m20, allrows=true, allcols=true))
describe(m20)

50×10 DataFrame
 Row │ n        k        alpha       errOld       errNew       timeOld      timeNew      normOld     normNew   normGen  
     │ Float64  Float64  Float64     Float64      Float64      Float64      Float64      Float64     Float64   Float64  
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    20.0      3.0   1.68642    6.57315e-9   0.00421566   0.00880007   0.0167261      2.29708   2.30511   2.29708
   2 │    95.0      9.0   0.0392704  1.63404e-10  7.12547e-12  0.00212912   0.00147745     1.74136   1.74136   1.74136
   3 │   818.0    631.0   0.213541   3.22431e-12  1.24945e-12  4.77454      3.33848        1.75715   1.75715   1.75715
   4 │   110.0     76.0   1.957      9.78674e-10  8.1329e-11   0.260404     0.144991       4.37756   4.37756   4.37756
   5 │    56.0     25.0  -0.826732   7.63893e-10  1.74486e-11  0.0172629    0.0112218      1.56313   1.56313   1.56313
   6 │   185.0     42.0  -

nothing

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,0.134594,6.6561e-13,4.54394e-10,1.61237,0,Float64
5,errNew,0.0705214,9.11195e-13,2.45653e-11,1.33373,0,Float64
6,timeOld,0.554626,0.000897013,0.0468974,7.87134,0,Float64
7,timeNew,0.380969,0.000341221,0.0202488,5.06729,0,Float64
8,normOld,14.8671,0.678741,2.87488,421.71,0,Float64
9,normNew,2.91624,0.608243,2.69964,9.42886,0,Float64


For distance at most $4\pi$. There are several examples where the log is better than the generating distance

In [28]:
# distance at most 4pi
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m40 = DataFrame(CSV.File("max_40.csv", header=false))
rename!(m40, header)
display(show(m40, allrows=true, allcols=true))
describe(m40)

50×10 DataFrame
 Row │ n        k        alpha       errOld       errNew       timeOld      timeNew       normOld     normNew   normGen  
     │ Float64  Float64  Float64     Float64      Float64      Float64      Float64       Float64     Float64   Float64  
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    20.0      3.0   1.68642    0.876103     0.0790723     0.135982     0.0189693       8.50409   4.48692   4.59415
   2 │    95.0      9.0   0.0392704  1.32006e-9   7.25082e-10   0.00742497   0.00986094      3.48271   3.48271   3.48271
   3 │   818.0    631.0   0.213541   5.61147e-12  6.05326e-12   7.2411       4.89437         3.51431   3.51431   3.51431
   4 │   110.0     76.0   1.957      0.562042     0.0256503     6.88965      2.96877      3143.02      8.73848   8.75513
   5 │    56.0     25.0  -0.826732   1.82116e-9   4.58851e-11   0.0623629    0.0273209       3.12626   3.12626   3.12626
   6 │   185.

nothing

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,0.298122,5.61147e-12,2.86051e-9,1.49669,0,Float64
5,errNew,0.0917439,1.86383e-12,0.0031078,1.54902,0,Float64
6,timeOld,1.74198,0.00171533,0.161948,19.1607,0,Float64
7,timeNew,1.29151,0.000716169,0.0535556,12.8026,0,Float64
8,normOld,255.505,1.13853,5.9207,3956.51,0,Float64
9,normNew,5.29282,1.08137,4.46827,21.5204,0,Float64
